In [2]:
import pandas as pd
import numpy as np
from collections import Counter
import os
from tqdm import tqdm as tqdm

# Data

In [3]:
# my_followers_df has been uploaded to Google Drive
followers = pd.read_csv('/Users/marzie/Documents/Data/my_followers_df')
followers

,user_id,podcaster_id
0,1022618,13480902
1,1022618,19399631
2,1022618,15572542
3,1022618,18150210
4,1022618,17363840
...,...,...
18578845,39261848,12347707
18578846,39261848,21673341
18578847,39261848,24438995
18578848,39261848,19624777


### drop users who follow fewer than 10 podcasts. 

In [4]:
sub_num_min = 10 

In [5]:
# Find out how many podcasts each user follows.

follower_df_user_id_s = followers['user_id'].tolist()

user_sub = dict(Counter(follower_df_user_id_s))

following_df = pd.DataFrame(
    data=list(user_sub.items()),
    columns=['user_id', 'following']).set_index('user_id')

following_df.head()

,following
user_id,
1022618,203
1032649,28
1038639,99
1049063,57
1049719,35


In [6]:
# drop users who follow fewer than 10 podcasts.

updated_users = following_df[following_df['following'] > sub_num_min].index.values

print(updated_users)
print()
print(len(updated_users))

[ 1022618  1032649  1038639 ... 39247460 39261381 39261848]

479210


### Split users into train and test.

The cell below was run once, and then we used the saved result.

In [7]:
# # Split updated_users into random train and test subsets.
# # %80 train and %20 test.

# np.random.shuffle(updated_users)

# train_user_ids = updated_users[:int(len(updated_users)*0.8)]
# test_user_ids = updated_users[int(len(updated_users)*0.8):]

# #save train and test user_ids
# np.save("train_user_ids", train_user_ids)
# np.save("test_user_ids", test_user_ids)

In [8]:
train_user_ids = np.load("train_user_ids.npy")
test_user_ids = np.load("test_user_ids.npy")

### Finding the co-following matrix based on train users.

In [9]:
df = followers[followers['user_id'].isin(train_user_ids)]

In [10]:
matrix_rows = df['podcaster_id'].unique()

matrix_columns = df['user_id'].unique()

In [11]:
# Construct the user-podcast Biadjacency Matrix

matrix = np.zeros((len(matrix_rows), len(matrix_columns)), dtype=bool)

for i in tqdm(range(len(matrix_rows))):
    
    mask = (df['podcaster_id'] == matrix_rows[i])
    
    index = np.where(np.in1d(matrix_columns, np.array(df[mask]['user_id'])))[0]
    
    matrix[i][index] = True

100%|██████████████████████████████████████| 3232/3232 [00:26<00:00, 123.22it/s]


In [12]:
# Construct common_followers Matrix

number_of_podcasters = len(matrix_rows)

common_followers = np.zeros((number_of_podcasters, number_of_podcasters))

for i in tqdm(range(number_of_podcasters)):
    for j in range(i+1, number_of_podcasters):
        
        common_followers[i, j] = (matrix[i]*matrix[j]).sum()

100%|███████████████████████████████████████| 3232/3232 [22:57<00:00,  2.35it/s]
